In [4]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# Instance Class

In [5]:
class Instance:
    O: int  # number of requests
    I: int  # number of items
    A: int  # number of corridors

    u_oi: np.ndarray  # quantity of each item for each request
    u_ai = np.ndarray  # quantity of each item in each corridor

    LB: int  # lower bound on the wave size
    UB: int  # upper bound on the wave size

    def __init__(self, input_file):
        with open(input_file, 'r') as file:
            self.O, self.I, self.A = map(int, file.readline().split())

            # Read orders matrix:
            self.u_oi = np.zeros((self.O, self.I), dtype=int)
            for i in range(self.O):
                data = np.fromstring(file.readline(), dtype=int, sep=' ')
                indices, qtys = data[1::2], data[2::2]
                self.u_oi[i, indices] = qtys

            # Read corridors matrix:
            self.u_ai = np.zeros((self.A, self.I), dtype=int)
            for i in range(self.A):
                data = np.fromstring(file.readline(), dtype=int, sep=' ')
                indices, qtys = data[1::2], data[2::2]
                self.u_ai[i, indices] = qtys

            # Read lower and upper bounds for wave size:
            self.LB, self.UB = map(int, file.readline().split())


input_file = "datasets/a/instance_0020.txt"
inst = Instance(input_file)

# Linearized model

In [6]:
# Create a new model
n = gp.Model("meli-linearized")

## Variables:
d = n.addVar(vtype=GRB.CONTINUOUS, lb=1 / inst.A, ub=1, name='d')  # the denominator of the objective cost function
θ = n.addMVar(inst.O, vtype=GRB.CONTINUOUS, lb=0, ub=1, name='θ')  # wave selected requests x d
α = n.addMVar(inst.A, vtype=GRB.CONTINUOUS, lb=0, ub=1, name='α')  # wave visited corridors x d
n.update()

## Restrictions:
wave_size_ = (inst.u_oi.T @ θ).sum()

# Operational limits on the total number of items for the orders included in the wave:
n.addConstr(wave_size_ >= inst.LB * d)
n.addConstr(wave_size_ <= inst.UB * d)

# The selected corridors have sufficient storage for each of the items within the wave:
n.addConstrs(θ @ inst.u_oi[:, i] <= α @ inst.u_ai[:, i] for i in range(inst.I))

wave_corridors_ = α.sum()  # number of used corridors x d
n.addConstr(wave_corridors_ == 1)  # equivalent to d = 1/wave_corridors_
n.update()

## Objective function:
n.setObjective(wave_size_, GRB.MAXIMIZE)

## Model solving:
dX = 1
try:
    n.optimize()

    # Print the found solution:
    dX = d.X
    for v in n.getVars():
        if v.VarName == 'd': continue
        print(f"{v.VarName} {v.X / dX:g}")
    print(f"Obj {wave_size_.getValue() / dX:g}")

    print(f"\nRequests, Items, Corridors: {inst.O}, {inst.I}, {inst.A}")
    print(f"Wave size: {wave_size_.getValue():g}")
    print(f"Number of used corridors: {wave_corridors_.getValue() / dX:g}")
except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")
except AttributeError:
    print("Encountered an attribute error")

Set parameter Username
Set parameter LicenseID to value 2615956
Academic license - for non-commercial use only - expires 2026-01-28
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Freedesktop SDK 23.08 (Flatpak runtime)")

CPU model: AMD Ryzen 7 8845HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 8 rows, 11 columns and 47 nonzeros
Model fingerprint: 0xefbbab1c
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 5e+00]
  Bounds range     [2e-01, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 8 rows, 11 columns, 47 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2000000e+01   5.750000e+00   0.000000e+00      0s
       9    5.9411765e+00   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.9411764

# Full model

In [7]:
%%script echo skipping

# Create a new model
m = gp.Model("meli-nonlinear")

## Variables:
O_ = m.addMVar(inst.O, vtype=GRB.BINARY, name='o')  # wave selected requests
A_ = m.addMVar(inst.A, vtype=GRB.BINARY, name='a')  # wave visited corridors
obj = m.addVar(vtype=GRB.CONTINUOUS, lb=1, ub=inst.u_oi.sum(), name='Obj')  # the objective cost function
m.update()

## Restrictions:
wave_size = (inst.u_oi.T @ O_).sum()  # total number of items in the wave

# Operational limits on the total number of items for the orders included in the wave:
m.addConstr(wave_size >= inst.LB)
m.addConstr(wave_size <= inst.UB)

# The selected corridors have sufficient storage for each of the items within the wave:
m.addConstrs(O_ @ inst.u_oi[:, i] <= A_ @ inst.u_ai[:, i] for i in range(inst.I))

m.addConstr(obj <= wave_size)  # basic cut
m.addConstr(obj <= wave_size_.getValue() / dX)  # linear relaxation UB
m.update()

## Objective function:
wave_corridors = A_.sum()  # number of used corridors
m.addConstr(obj == wave_size / wave_corridors)
m.setObjective(obj, GRB.MAXIMIZE)

## Model solving:
try:
    m.optimize()

    # Print the found solution:
    for v in m.getVars():
        print(f"{v.VarName} {v.X:g}")

    print(f"\nRequests, Items, Corridors: {inst.O}, {inst.I}, {inst.A}")
    print(f"Wave size: {wave_size.getValue():g}")
    print(f"Number of used corridors: {wave_corridors.getValue():g}")
except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")
except AttributeError:
    print("Encountered an attribute error")

skipping


# Binary search approach:

In [8]:
# Create a new model
s = gp.Model("meli-search")

## Variables:
O_ = s.addMVar(inst.O, vtype=GRB.BINARY, name='o')  # wave selected requests
A_ = s.addMVar(inst.A, vtype=GRB.BINARY, name='a')  # wave visited corridors
obj = s.addVar(vtype=GRB.CONTINUOUS, lb=1, ub=inst.u_oi.sum(), name='Obj')  # the objective cost function
s.update()

## Restrictions:
wave_size = (inst.u_oi.T @ O_).sum()  # total number of items in the wave

# Operational limits on the total number of items for the orders included in the wave:
s.addConstr(wave_size >= inst.LB)
s.addConstr(wave_size <= inst.UB)

# The selected corridors have sufficient storage for each of the items within the wave:
s.addConstrs(O_ @ inst.u_oi[:, i] <= A_ @ inst.u_ai[:, i] for i in range(inst.I))

s.addConstr(obj <= wave_size)  # basic cut
s.addConstr(obj <= wave_size_.getValue() / dX)  # linear relaxation UB
s.update()

## Objective function:
wave_corridors = A_.sum()  # number of used corridors
s.addConstr(obj == wave_size / wave_corridors)
s.setObjective(obj, GRB.MAXIMIZE)

## Model solving:
try:
    s.optimize()

    # Print the found solution:
    for v in s.getVars():
        print(f"{v.VarName} {v.X:g}")

    print(f"\nRequests, Items, Corridors: {inst.O}, {inst.I}, {inst.A}")
    print(f"Wave size: {wave_size.getValue():g}")
    print(f"Number of used corridors: {wave_corridors.getValue():g}")
except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")
except AttributeError:
    print("Encountered an attribute error")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Freedesktop SDK 23.08 (Flatpak runtime)")

CPU model: AMD Ryzen 7 8845HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 9 rows, 11 columns and 47 nonzeros
Model fingerprint: 0xfee7db49
Model has 1 general nonlinear constraint (1 nonlinear terms)
Variable types: 1 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [5e+00, 1e+01]
Presolve model has 1 nlconstr
Added 2 variables to disaggregate expressions.
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 14 rows, 14 columns, 68 nonzeros
Presolved model has 1 bilinear constraint(s)

Solving non-convex MIQCP

Variable types: 2 continuous, 12 integer (10 binary)

Root relaxation: objective 5.940570e+00, 19 iterations, 